# Ejemplo básico con TensorFlow 2.0

En este ejercicio vamos a recrear nuestro algoritmo de aprendizaje utilizando TF 2.

## Importar las librerías relevantes

In [ ]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow as tf

## Generacióm de datos

Generarémos los datos de la misma manera que lo hicimos en el ejercicio annterior.  La única diferencia es que ahora guardamos los datos en un archivo *.npz*.  NPZ es el tipo de archivo propio de NumPy que permite guardar los arreglos NumPy.  Introducimos este cambio porque en el ML a menudo:

* nos dan datos (csv, base de datos, etc.)
* preprocesamos los datos y los dejamos en un formato deseado (veremos este tema después)
* se guardan los datos en archivos npz (si es que estamos trabajando con Python) para uso posterior

No hay nada especial de todo esto.  Solo guardamos arreglos NumPy en un archivo re-utlizable.

Trabajaremos con dos variables de entrada, las x1 y x2. Se generan al azar a partir de una distribución uniforme.

Se creará una matriz con estas dos variables.  La matriz X del modelo lineal y = x * w + b

In [ ]:
# Por facilidad, declaramos una variable que indique el tamaño del conjunto 
#      de datos de entrenamiento.
observaciones = 1000

x1 = np.random.uniform(low=-10, high=10, size=(observaciones,1))
x2 = np.random.uniform(-10, 10, (observaciones,1))

entradas_generadas = np.column_stack((x1,x2))



### Generar las metas a las que debemos apuntar

Inventaremos una función f(x1, x2) = 2 * x1 - 3 * x2 + 5 + <ruido pequeño>.  El ruido es para hacerlo más realista.

En esencia estamos diciendo que los pesos serán 2 y -3, y es sesgo es 5

Utilizaremos la metodología de ML, y veremos si el algoritmo la ha aprendido. 

In [ ]:
ruido = np.random.uniform(-1, 1, (observaciones,1))

targets_generados = 2 * x1 - 3 * x2 + 5 + ruido

Hasta ahora todo ha sido igual.  El siguiente paso sí es nuevo y es que estaremos guardando la información en un archivo *.npz* que llamaremos "Datos_TF)

In [ ]:
np.savez('Datos_TF', entradas = entradas_generadas, targets = targets_generados)

## Resolver con TensorFlow

<i/>Nota: Esta introducción de TensorFloe es muy básica.  El TF tiene muchas más capacidaded y profundidad que esto.<i>

In [ ]:
# Vamos a cargar los datos desde el archivo NPZ.  Por supuesto, esto no era necesario acá
datos_entrenamiento = np.load('Datos_TF.npz')

En el ejercicio anterior tuvimos que dar valores iniciales, acá solo damos los tamaños

In [ ]:
tamanio_entrada = 2   # el número de variables que tenemos

tamanio_salida = 1   # el número de salidas que tenemos

Delineamos o esbozamos el modelo.  

Se hace con **"Sequential"**

Notese que no se pide cálculo alguno - solo describimos nuestra red

Acá se debe listar cada capa "layer"

El método **"Dense"** indica, que nuestra operación matemática será (xw + b)

Hay otros parámetros que se pueden incluir para particularizar el modelo, en nuestro caso solo estamos tratando de crear una solución que sea tan parecida a la de nuestro modelo NumPy-

In [ ]:
model = tf.keras.Sequential([
                           
                            tf.keras.layers.Dense(tamanio_salida,
                                                 kernel_initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1),
                                                 bias_initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1)
                                                 )
                            ])

También podemos definir un optimizador a la medida, donde podemos especificar la tasa de aprendizaje.

In [ ]:
optimizador_adhoc = tf.keras.optimizers.SGD(learning_rate=0.02)

Es posible que se necesite una función de pérdida a la medida.  Eso es mucho más difícil implementar y no trabajaremos esto en el curso.

**"compile"** es donde podemos seleccionar e indicar los optimizadores y la pérdida.

In [ ]:
model.compile(optimizer = optimizador_adhoc, loss='mean_squared_error')

Finalmente ajustamos el modelo, indicando las entradas y los targets.  

En vez de usar el término *iteraciones*, se utiliza el término *épocas*.  Si no se especifica el número de épocas este será 1 (una sola época de entrenamiento), así que este número es algo obligatorio.

El parámetro **"verbose"** se refiere a cuánta información queremos que despliegue durante la ejecución.  Se vale probar diferentes números...uno que es bastante bueno es el 2

In [ ]:
model.fit(datos_entrenamiento['entradas'], datos_entrenamiento['targets'], epochs = 100, verbose = 2)

## Extracción de los pesos y sesgos

La exctracción de el(los) peso(s) y sesgo(s) de un modelo no es un paso esencial para el proceso de ML.  De hecho, en un contexto de aprendizaje profundo, no nos daría mucha información útil.  Sin embargo este ejemplo simple se armó de tal forma que nos permite verificar si las respueestas que obtenemos son correctas.

In [ ]:
model.layers[0].get_weights()    # el cero (0) es porque solo tenemos una capa

Podemos almacenar los pesos y los sesgos en variables diferentes para facilitar la revisión.

OJO!   Pueden haber cientos o miles de estos!!!!

In [ ]:
pesos = model.layers[0].get_weights()[0]
pesos

In [ ]:

sesgos = model.layers[0].get_weights()[1]
sesgos

## Extraer las salidas y hacer predicciones

Una vez más, este no es un paso esencial, sin embargo, generalmente vamos a querer hacer predicciones.

Podemos predecir nuevos valores para hacer uso del modelo.  A veces es útil redondear los valores para que la salida sea más legible.  

Generalmente se utiliza este método con DATOS NUEVOS, en vez de usar los datos de entrenamiento originales.

In [ ]:
model.predict_on_batch(datos_entrenamiento['entradas']).round(1)

Si desplegamos nuestras metas (valores reales), podemos compararlas manualmente con las predicciones.

In [ ]:
datos_entrenamiento['targets'].round(1)

## Graficar los datos

El modelo está ya optimizado, por lo que las salidas se han calculado sobre la última forma, o estado, del modelo.

Necesitamos comprimir o empacar **"squeeze"** los arreglos para dejarlos en un formato que es el esperado por la función graficadora.  No cambia nada ya que dejamos dimensiones de tamaño 1 - solo es un tecnisismo

In [ ]:
plt.plot(np.squeeze(model.predict_on_batch(datos_entrenamiento['entradas'])), 
         np.squeeze(datos_entrenamiento['targets']))
plt.xlabel('salidas')
plt.ylabel('targets')
plt.show()

In [ ]:
fig = px.scatter(x = np.squeeze(model.predict_on_batch(datos_entrenamiento['entradas'])), 
                 y =  np.squeeze(datos_entrenamiento['targets']))

fig.update_layout(
    title="Comparación predicciones vrs metas",
    xaxis_title="Salidas (Predicciones)",
    yaxis_title="Targets (Metas)",
    width = 600,
    height = 400,)

fig.show()

Listo, lo que vemos debe ser exactamente igual a lo que vimos en el ejercicio pasado!

A estas alturas quizás no le vean la gracia al TensorFlow.  En términos de líneas código es igual al del ejercicio con NumPy para llegar al mismo resultado.  Sin embargo, a medida que profundizemos en el tema, veremos que TensorFlow nos ahorrará cientos de líneas de código.